## NDC - ATC - DrugBank_ID

### MIMIC3_NDC - ATC (CANNOT match those drugs without NDC codes)

In [6]:
## Import base packages
import numpy as np
import pandas as pd
from tools import *
import re
# import requests
from os.path import join

read_prefix = '/data/MIMIC3/'
write_prefix = '/data/liu/adverse_events'


In [7]:
def get_ndc_codes(ndc):
    
    if(ndc == '0'): return []
    
    ndc_codes = []
    if(ndc[0]=='0'):
        ## 442
        ndc_codes = ndc_codes + [('-').join([ndc[1:5],ndc[5:9],ndc[9:]])]
    if(ndc[5]=='0'):
        ## 532
        ndc_codes = ndc_codes + [('-').join([ndc[0:5],ndc[6:9],ndc[9:]])]
    if(ndc[9]=='0'):
        ## 541
        ndc_codes = ndc_codes + [('-').join([ndc[0:5],ndc[5:9],ndc[-1]])]
    return ndc_codes

In [9]:
## Read raw prescriptions
usecols=["SUBJECT_ID", "STARTDATE", "ENDDATE","DRUG","NDC","DRUG_NAME_GENERIC","ROUTE"]
pres_df = pd.read_csv(join(read_prefix,'PRESCRIPTIONS.csv'),usecols=usecols,dtype={'NDC': str})
pres_df.head()

,SUBJECT_ID,STARTDATE,ENDDATE,DRUG,DRUG_NAME_GENERIC,NDC,ROUTE
0,6,2175-06-11 00:00:00,2175-06-12 00:00:00,Tacrolimus,Tacrolimus,00469061711,PO
1,6,2175-06-11 00:00:00,2175-06-12 00:00:00,Warfarin,Warfarin,00056017275,PO
2,6,2175-06-11 00:00:00,2175-06-12 00:00:00,Heparin Sodium,NaN,00338055002,IV
3,6,2175-06-11 00:00:00,2175-06-12 00:00:00,D5W,NaN,0,IV
4,6,2175-06-11 00:00:00,2175-06-12 00:00:00,Furosemide,Furosemide,00054829725,PO


In [10]:
## Drop duplicate NDC
mimic_drugs_NDCs = pres_df[['NDC']].dropna().drop_duplicates()

In [34]:
## 4204 ndcs, transform them to 3 kinds of possible formatted codes
mimic_drugs_NDCs['NDC_CODES'] = mimic_drugs_NDCs['NDC'].apply(get_ndc_codes)
mimic_drugs_NDCs['L'] = mimic_drugs_NDCs['NDC'].apply(len)
mimic_drugs_NDCs = mimic_drugs_NDCs[mimic_drugs_NDCs['L']==11]
mimic_drugs_NDCs.head()

,NDC,NDC_CODES,L
0,00469061711,"[0469-0617-11, 00469-617-11]",11
1,00056017275,"[0056-0172-75, 00056-172-75]",11
2,00338055002,"[0338-0550-02, 00338-550-02, 00338-0550-2]",11
4,00054829725,[0054-8297-25],11
5,00056016975,"[0056-0169-75, 00056-169-75]",11


In [35]:
## Split three kinds codes into three columns
mimic_drugs_NDCs= pd.concat([mimic_drugs_NDCs, mimic_drugs_NDCs['NDC_CODES'].apply(pd.Series)],axis=1)
mimic_drugs_NDCs.head()

,NDC,NDC_CODES,L,0,1,2
0,00469061711,"[0469-0617-11, 00469-617-11]",11,0469-0617-11,00469-617-11,NaN
1,00056017275,"[0056-0172-75, 00056-172-75]",11,0056-0172-75,00056-172-75,NaN
2,00338055002,"[0338-0550-02, 00338-550-02, 00338-0550-2]",11,0338-0550-02,00338-550-02,00338-0550-2
4,00054829725,[0054-8297-25],11,0054-8297-25,NaN,NaN
5,00056016975,"[0056-0169-75, 00056-169-75]",11,0056-0169-75,00056-169-75,NaN


In [36]:
## remove intermediate result columns AND Convert 3 NDC_CODE columns to rows AND drop null NDC_CODE:
mimic_drugs_NDCs = mimic_drugs_NDCs[['NDC', 0, 1, 2]]
mimic_drugs_NDCs=pd.melt(mimic_drugs_NDCs, id_vars=["NDC"], \
                         value_name="NDC_CODE").sort_values(by=['NDC','variable'])
mimic_drugs_NDCs=mimic_drugs_NDCs.dropna(subset=['NDC_CODE'])
mimic_drugs_NDCs.head()

,NDC,variable,NDC_CODE
3550,00002050101,0,0002-0501-01
7753,00002050101,1,00002-501-01
11956,00002050101,2,00002-0501-1
3672,00002140701,0,0002-1407-01
7875,00002140701,1,00002-1407-1


In [37]:
## save mimic NDC-CODE to file
aa.write2file(mimic_drugs_NDCs,'mimic_NDCcode')

In [39]:
## read NDC-ATC data
drugbank_path = '/data/liu/adverse_events/mapping/%s'
ndc2atc_df = pd.read_csv(drugbank_path%'ndc_atc4_201903.csv',sep=',',dtype={'RXCUI':str})
ndc2atc_df = ndc2atc_df.rename(columns = {'NDC':'NDC_CODE'})
ndc2atc_df.head()

,YEAR,MONTH,NDC_CODE,RXCUI,ATC4
0,2015,5,0002-7712-27,1652242,A10AD
1,2017,10,0002-7712-61,1652242,A10AB
2,2017,10,0002-7712-61,1652242,A10AC
3,2017,10,0002-7712-61,1652242,A10AD
4,2015,5,0002-7712-99,1652242,A10AB


In [40]:
## Join mimic_drugs_NDCs to drugbank_df(NDC - ATC4), Inner_join no null value will be generated for ATC
mimic_drug_ATCs = aa.inner_join(mimic_drugs_NDCs, ndc2atc_df, 'NDC_CODE')
mimic_drug_ATCs.head()

,NDC,variable,NDC_CODE,YEAR,MONTH,RXCUI,ATC4
0,00002821501,0,0002-8215-01,1983,6,311036,A10AB
1,00002821501,0,0002-8215-01,1983,6,311036,A10AC
2,00002821501,0,0002-8215-01,1983,6,311036,A10AD
3,00002821501,0,0002-8215-01,1983,6,311036,A10AE
4,00002821501,0,0002-8215-01,1983,6,311036,A10AF


In [46]:
## select columns of mimic_atc and write it into files
mimic_atc_cols = ['NDC', 'NDC_CODE', 'ATC4']
mimic_drug_ATCs = mimic_drug_ATCs[mimic_atc_cols]
aa.write2file(mimic_drug_ATCs, 'mimic_ATCcode')

###  ATC to drugbank_ID

#### read drug atc-drugbank_id from drugbank.tsv

In [2]:
drugbank_df = aa.read_data(filename='mapping/drugbank',sep='\t',suffix='.tsv')

In [18]:
## drugbank: atc_codes -> ATC4
drugbank_col = ['drugbank_id','atc_codes']
drugbank_df = drugbank_df[drugbank_col]
drugbank_df['ATC4'] = drugbank_df['atc_codes'].apply(lambda x: str(x)[:5])
drugbank_df = drugbank_df[['drugbank_id','ATC4']].drop_duplicates()
drugbank_df.head()

,drugbank_id,ATC4
0,DB00001,B01AE
1,DB00002,L01XC
2,DB00003,R05CB
3,DB00004,L01XX
4,DB00005,L04AB


In [19]:
drug_mimic_atcs = pd.read_csv('/data/liu/adverse_events/mimic_ATCcode.csv',dtype={'NDC':str})
drug_mimic_atcs.head()

,NDC,NDC_CODE,ATC4
0,00002821501,0002-8215-01,A10AB
1,00002821501,0002-8215-01,A10AC
2,00002821501,0002-8215-01,A10AD
3,00002821501,0002-8215-01,A10AE
4,00002821501,0002-8215-01,A10AF


In [20]:
drug_mimic_bankids = aa.inner_join(drug_mimic_atcs, drugbank_df, 'ATC4')
drug_mimic_bankids.head()

,NDC,NDC_CODE,ATC4,drugbank_id
0,00002821501,0002-8215-01,A10AB,DB00030
1,00002821501,0002-8215-01,A10AB,DB01309
2,00002850101,0002-8501-01,A10AB,DB00030
3,00002850101,0002-8501-01,A10AB,DB01309
4,00002871501,0002-8715-01,A10AB,DB00030


###  SIDER4 drugbank_ID -> Match MIMIC3 and SIDER4

In [12]:
## read data of  side effects
side_effects = aa.read_data('SIDER4/side-effects',sep='\t',suffix='.tsv',pre='/data/liu/adverse_events')
side_effects.head()

,drugbank_id,drugbank_name,umls_cui_from_meddra,side_effect_name
0,DB07768,"(10ALPHA,13ALPHA,14BETA,17ALPHA)-17-HYDROXYAND...",C0000729,Abdominal cramps
1,DB07768,"(10ALPHA,13ALPHA,14BETA,17ALPHA)-17-HYDROXYAND...",C0000737,Abdominal pain
2,DB07768,"(10ALPHA,13ALPHA,14BETA,17ALPHA)-17-HYDROXYAND...",C0232492,Abdominal pain upper
3,DB07768,"(10ALPHA,13ALPHA,14BETA,17ALPHA)-17-HYDROXYAND...",C0740651,Abdominal symptom
4,DB07768,"(10ALPHA,13ALPHA,14BETA,17ALPHA)-17-HYDROXYAND...",C0877331,Abnormal clotting factor


In [28]:
ndc_side_effects = aa.inner_join(drug_mimic_bankids[['NDC','drugbank_id']],
                                side_effects[['drugbank_id','umls_cui_from_meddra']],'drugbank_id')
ndc_side_effects.head()

,NDC,drugbank_id,umls_cui_from_meddra
0,00003029305,DB00620,C0000731
1,00003029305,DB00620,C1302752
2,00003029305,DB00620,C0000833
3,00003029305,DB00620,C0702166
4,00003029305,DB00620,C0175167


In [29]:
aa.write2file(ndc_side_effects, "ndc-side-effects")

## umls_cui - NOMED code - ICD9

In [11]:
ndc_side_effects = pd.read_csv('/data/liu/adverse_events/ndc-side-effects.csv',dtype={'NDC':str})
ndc_side_effects.head()

,NDC,drugbank_id,umls_cui_from_meddra
0,00003029305,DB00620,C0000731
1,00003029305,DB00620,C1302752
2,00003029305,DB00620,C0000833
3,00003029305,DB00620,C0702166
4,00003029305,DB00620,C0175167


In [12]:
diagnosis_df = read_data(join('/data/MIMIC3','D_ICD_DIAGNOSES'))
diagnosis_df.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


In [19]:
icd9_SNOMED =pd.read_csv('/data/liu/adverse_events/mapping/ICD9CM_SNOMED_MAP_1TO1_201812.txt'
                         ,sep='\t', dtype={'ICD9_CODE':str, 'SNOMED_CID':str})
icd9_SNOMED[:1]

,ICD_CODE,ICD_NAME,IS_CURRENT_ICD,IP_USAGE,OP_USAGE,AVG_USAGE,IS_NEC,SNOMED_CID,SNOMED_FSN,IS_1-1MAP,CORE_USAGE,IN_CORE
0,427.31,Atrial fibrillation,1,1.89778,3.20644,2.55211,0,49436004,Atrial fibrillation (disorder),1,0.499,1


In [17]:
icd9_SNOMEDOM =pd.read_csv('/data/liu/adverse_events/mapping/ICD9CM_SNOMED_MAP_1TOM_201812.txt'
                         ,sep='\t', dtype={'ICD9_CODE':str, 'SNOMED_CID':str})
icd9_SNOMEDOM[:1]

,ICD_CODE,ICD_NAME,IS_CURRENT_ICD,IP_USAGE,OP_USAGE,AVG_USAGE,IS_NEC,SNOMED_CID,SNOMED_FSN,IS_1-1MAP,CORE_USAGE,IN_CORE
0,V76.12,Other screening mammogram,1,NaN,2.88756,1.44378,1.0,NaN,NaN,0,NaN,0


In [20]:
icd9_SNOMED = icd9_SNOMED[['ICD_CODE','SNOMED_CID']].dropna(subset=['SNOMED_CID'])
icd9_SNOMED['ICD9_CODE']=icd9_SNOMED['ICD_CODE'].apply(lambda x: str(x).replace('.',''))
icd9_SNOMED.head()

,ICD_CODE,SNOMED_CID,ICD9_CODE
0,427.31,49436004,42731
1,599.0,68566005,5990
2,486,233604007,486
3,585.6,46177005,5856
4,414.01,1641000119107,41401


In [21]:
icd9_SNOMEDOM = icd9_SNOMEDOM[['ICD_CODE','SNOMED_CID']].dropna(subset=['SNOMED_CID'])
icd9_SNOMEDOM['ICD9_CODE']=icd9_SNOMEDOM['ICD_CODE'].apply(lambda x: str(x).replace('.',''))
icd9_SNOMEDOM.head()

,ICD_CODE,SNOMED_CID,ICD9_CODE
1,V57.89,305839006,V5789
2,V57.89,305840008,V5789
3,V57.89,305844004,V5789
4,V57.89,305842000,V5789
5,V57.89,305843005,V5789


In [28]:
icd9_SNOMED = pd.concat((icd9_SNOMED, icd9_SNOMEDOM), axis=0)
icd9_SNOMED.head()

,ICD_CODE,SNOMED_CID,ICD9_CODE
0,427.31,49436004,42731
1,599.0,68566005,5990
2,486,233604007,486
3,585.6,46177005,5856
4,414.01,1641000119107,41401


In [30]:
cui_SNOMED =pd.read_csv('/data/liu/adverse_events/mapping/SNOMEDCT_CORE_SUBSET_201901.txt'
                        ,sep='|', dtype={'SNOMED_CID':str})
print(len(cui_SNOMED))
cui_SNOMED[:1]

6482


,SNOMED_CID,SNOMED_FSN,SNOMED_CONCEPT_STATUS,UMLS_CUI,OCCURRENCE,USAGE,FIRST_IN_SUBSET,IS_RETIRED_FROM_SUBSET,LAST_IN_SUBSET,REPLACED_BY_SNOMED_CID
0,38341003,"Hypertensive disorder, systemic arterial (diso...",Current,C0020538,8.0,3.2242,200907,False,NaN,NaN


In [31]:
cui_SNOMED=cui_SNOMED[['SNOMED_CID','UMLS_CUI']]
cui_SNOMED[:1]

,SNOMED_CID,UMLS_CUI
0,38341003,C0020538


In [33]:
icd9_cui = inner_join(icd9_SNOMED, cui_SNOMED,'SNOMED_CID')
print(len(icd9_cui))
icd9_cui[:1]

4387


,ICD_CODE,SNOMED_CID,ICD9_CODE,UMLS_CUI
0,427.31,49436004,42731,C0004238


In [34]:
ndc_side_effects = pd.read_csv('/data/liu/adverse_events/ndc-side-effects.csv',
                              dtype={'NDC':str})
ndc_side_effects.head()

,NDC,drugbank_id,umls_cui_from_meddra
0,00003029305,DB00620,C0000731
1,00003029305,DB00620,C1302752
2,00003029305,DB00620,C0000833
3,00003029305,DB00620,C0702166
4,00003029305,DB00620,C0175167


In [35]:
ndc_side_effects = ndc_side_effects.rename(columns={'umls_cui_from_meddra':'UMLS_CUI'})
ndc_side_effects.columns

Index(['NDC', 'drugbank_id', 'UMLS_CUI'], dtype='object')

In [36]:
ndc_idc9_side_effects = inner_join(ndc_side_effects, icd9_cui, 'UMLS_CUI')
ndc_idc9_side_effects.head()

,NDC,drugbank_id,UMLS_CUI,ICD_CODE,SNOMED_CID,ICD9_CODE
0,00003029305,DB00620,C0000731,789.36,60728008,78936
1,00003049420,DB00620,C0000731,789.36,60728008,78936
2,00009001104,DB00620,C0000731,789.36,60728008,78936
3,00009001201,DB00620,C0000731,789.36,60728008,78936
4,00009003101,DB00620,C0000731,789.36,60728008,78936


In [37]:
write2file(ndc_idc9_side_effects, join(write_prefix,'ndc_icd9_side_effects'))

## Match to side effects in mimic3

In [2]:
sider_code = pd.read_csv('/data/liu/adverse_events/ndc_icd9_side_effects.csv',dtype={'NDC':str,'ICD_CODE':str})
sider_code = sider_code[['NDC','ICD9_CODE']].drop_duplicates()
sider_code.head()

/home/liu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,NDC,ICD9_CODE
0,00003029305,52120
1,00003049420,52120
2,00009001104,52120
3,00009001201,52120
4,00009003101,52120


In [3]:
prescriptions = pd.read_csv('/data/MIMIC3/PRESCRIPTIONS.csv',dtype={'NDC':str})
prescriptions = prescriptions[['SUBJECT_ID','NDC','DRUG']].drop_duplicates()
prescriptions.head()

/home/liu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SUBJECT_ID,NDC,DRUG
0,6,00469061711,Tacrolimus
1,6,00056017275,Warfarin
2,6,00338055002,Heparin Sodium
3,6,0,D5W
4,6,00054829725,Furosemide


In [5]:
sider_pres = aa.inner_join(prescriptions,sider_code,'NDC')
aa.write2file(sider_pres, 'pres_sider')
sider_pres.head()

,SUBJECT_ID,NDC,DRUG,ICD9_CODE
0,6,00469061711,Tacrolimus,71690
1,6,00469061711,Tacrolimus,71990
2,6,00469061711,Tacrolimus,7993
3,6,00469061711,Tacrolimus,7245
4,6,00469061711,Tacrolimus,3482


## Match to diagnosis data

In [ ]:
sider_pres = aa.read_data('pres_sider')

In [6]:
## dianosis dataframe with diagnosis name
diagnosis = aa.read_data('DIAGNOSES_ICD',pre='/data/MIMIC3')
diagnosis = diagnosis[['SUBJECT_ID','ICD9_CODE']]
diagnosis_code = aa.read_data('D_ICD_DIAGNOSES',pre='/data/MIMIC3')
diagnosis_code = diagnosis_code[['ICD9_CODE','SHORT_TITLE']]
diagnosis_name = aa.inner_join(diagnosis,diagnosis_code,'ICD9_CODE')
diagnosis_name = diagnosis_name.drop_duplicates()
diagnosis_name.head()

,SUBJECT_ID,ICD9_CODE,SHORT_TITLE
0,109,40301,Mal hyp kid w cr kid V
29,384,40301,Mal hyp kid w cr kid V
31,518,40301,Mal hyp kid w cr kid V
34,948,40301,Mal hyp kid w cr kid V
35,1195,40301,Mal hyp kid w cr kid V


In [ ]:
sider_pres_diag = pd.read_csv('/data/MIMIC3/PRESCRIPTIONS.csv',dtype={'NDC':str})aa.inner_join(sider_pres,diagnosis_name,'')
sider_pres_diag.head()

In [ ]:
aa.write2file(sider_pres_diag, 'sider_pres_diag')